# YOLOv8

## Instalação e Configuração

In [ ]:
# Instalando o pacote do YOLO v8
!pip install ultralytics

In [ ]:
import ultralytics
import cv2

In [ ]:
ultralytics.checks()

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

In [ ]:
!nvidia-smi

### Clonar repositório (alternativa)

In [ ]:
#!git clone https://github.com/ultralytics/ultralytics
#!pip install -r /content/ultralytics/requirements.txt
#!pip install -qe ultralytics

### Importação das demais bibliotecas

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from google.colab.patches import cv2_imshow

## Download do modelo

- Os modelos do YOLOv8 podem ser baixados aqui: 
https://github.com/ultralytics/assets/releases

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

## Realizando a detecção

A forma mais rápida de executar a deteçcão é através da CLI, conforme pode ver abaixo a sintaxe.

Os parâmetros:
* **task** - tarefa, podendo ser detecção, segmentação ou classificação
* **model** - o modelo que queremos usar. São disponibilizados 5 modelos: YOLOv8n (nano), YOLOv8s (small), YOLOv8m (medium), YOLOv8l (large), YOLOv8x (extra large). Nano é o mais rápido e o menor (menos pesado para rodar), enquanto que o Extra Large (YOLOv8x) é o mais preciso porém mais pesado para rodar, portanto será mais lento. 
  Para o valor desse parâmetro, basta informar o nome e ao lado .pt pois é a extensão do modelo treinado em pytorch. 

* **mode** - basicamente é o que queremos com o comando. queremos fazer a detecção/inferência/predição, portanto deixe =predict. Valores aceitos: [train, val, predict, export]

* **conf** - o limiar (threshold) que usaremos para filtrar detecções "fracas". Se a confiança estiver abaixo desse valor, não será considerada. Por padrão podemos configurar com um limiar bem baixo (ex: 0.25) e depois aumentar, caso verifique que detectou incorretamente algum objeto.  
* **source** - a imagem ou vídeo que queremos fazer a detecção.

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='cachorros00.jpeg' save=True

In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16,10)
  plt.axis('off')
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
resultado = cv2.imread('/content/runs/detect/predict/cachorros00.jpeg')
mostrar(resultado)

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://ultralytics.com/images/zidane.jpg' save=True

In [ ]:
resultado = cv2.imread('/content/runs/detect/predict2/zidane.jpg')
mostrar(resultado)

### Alterando o limiar (threshold) 

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.4 source='/content/italia.jpg' save=true

In [ ]:
resultado = cv2.imread('/content/runs/detect/predict3/italia.jpg')
mostrar(resultado)

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.6 source='/content/italia.jpg' save=true

In [ ]:
resultado = cv2.imread('/content/runs/detect/predict4/italia.jpg')
mostrar(resultado)

## Detecção em múltiplas imagens de uma vez

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.5 source='/content/imagens' save=true

In [ ]:
dir_resultados = '/content/runs/detect/predict5/'

In [ ]:
caminhos = [os.path.join(dir_resultados, f) for f in os.listdir(dir_resultados)]
print(caminhos)

In [ ]:
for caminho_imagem in caminhos:
  #print(caminho_imagem)
  imagem = cv2.imread(caminho_imagem)
  mostrar(imagem)

## Realizando a detecção - método através do Python

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
caminho_img = '/content/imagens/cachorros.jpg'
img = cv2.imread(caminho_img)
mostrar(img)

Para a deteção usaremos o método model.predict().

Ele aceita basicamente os mesmos parâmetros do método via CLI, que vimos nos exemplos mais acima.

* source -- A função aceita a imagem nos seguintes formatos: 
  * imagem em ndarray (usando o cv2.imread) ou PIL
  * diretório
  * caminho
  * URL
  * video
  * webcam (valor 0, que corresponde ao indice do dispositivo de câmera)

* save -- se desejar salvar a imagem no diretório predict deixei save=True. aqui nós vamos fazer de outra maneira, então não precisa setar esse parâmetro

* conf -- limiar de confiança 

* show -- Informe o parâmetro show=True para mostrar os valores das predições

* save_txt -- se deseja salvar em um txt os resultados

mais parâmetros https://docs.ultralytics.com/cfg/

In [ ]:
resultados = model.predict(source=img)

In [ ]:
resultados

In [ ]:
for r in resultados:
  print(r.boxes.boxes)

In [ ]:
for r in resultados:
  print(r.boxes.conf)

### Desenhando a caixa delimitadora sobre a imagem



In [ ]:
import funcoes_desenho

In [ ]:
resultado = funcoes_desenho.desenha_caixas(img, resultados[0].boxes.boxes)
mostrar(resultado)

## Teste com outros modelos 


In [ ]:
model = YOLO('yolov8m.pt')
caminho_img = '/content/imagens/italia.jpg'
img = cv2.imread(caminho_img)
resultados = model.predict(source = img, conf=0.6)
resultado_img = funcoes_desenho.desenha_caixas(img, resultados[0].boxes.boxes)
mostrar(resultado_img)

> Usando o YOLOv8x (extra large)

In [ ]:
model = YOLO('yolov8x.pt')
caminho_img = '/content/imagens/gato03.jpg'
img = cv2.imread(caminho_img)
resultados = model.predict(source = img, conf=0.6)
resultado_img = funcoes_desenho.desenha_caixas(img, resultados[0].boxes.boxes)
mostrar(resultado_img)

## Validação do modelo

In [ ]:
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')

In [ ]:
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

In [ ]:
!yolo task=detect mode=val model=yolov8x.pt data=coco128.yaml

## Detecção em vídeos

In [ ]:
!yolo task=detect mode=predict model=yolov8m.pt conf=0.5 source='/content/video_pessoas01.mp4' save=true

In [ ]:
import os
video_salvo = '/content/runs/detect/predict6/video_pessoas01.mp4'
video_final = '/content/video_pessoas01_resultado.mp4'
os.system(f"ffmpeg -i {video_salvo} -vcodec libx264 {video_final}")

In [ ]:
def exibir_video(video_caminho):
  from IPython.display import HTML
  from base64 import b64encode
  mp4 = open(video_caminho, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=500 controls>
    <source src="%s" type="video/mp4"
  </video>
  """ % data_url)

In [ ]:
exibir_video(video_final)

## Segmentação de imagens

### Via CLI

In [ ]:
!yolo task=segment mode=predict model=yolov8m-seg.pt conf=0.5 source='/content/imagens/cachorros.jpg' save=true

In [ ]:
resultado = cv2.imread('/content/runs/segment/predict/cachorros.jpg')
mostrar(resultado)

### Via Python

In [ ]:
model_seg = YOLO('yolov8m-seg.pt')
caminho_img = '/content/imagens/gato02.jpg'
img = cv2.imread(caminho_img)
resultados = model_seg.predict(source=img, conf=0.6)
resultado_img = funcoes_desenho.desenha_caixas(img, resultados[0].boxes.boxes)
mostrar(resultado_img)

In [ ]:
model_seg = YOLO('yolov8x-seg.pt')
caminho_img = '/content/imagens/person.jpg'
img = cv2.imread(caminho_img)
resultados = model_seg.predict(source=img, conf=0.6)
resultado_img = funcoes_desenho.desenha_caixas(img, resultados[0].boxes.boxes)
mostrar(resultado_img)

### Segmentação em vídeos

In [ ]:
!wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

In [ ]:
!yolo task=segment mode=predict model=yolov8m-seg.pt conf=0.5 source='/content/video_rua01.mp4' save=true

In [ ]:
video_salvo = "/content/runs/segment/predict2/video_rua01.mp4"
video_final = "/content/segmentacao_rua01.mp4" 

os.system(f"ffmpeg -i {video_salvo} -vcodec libx264 {video_final}")

In [ ]:
exibir_video(video_final)